In [1]:
import os

In [2]:
# Confirm the current working directory
%pwd

In [3]:
# Change the directory to one level below the current working directory
os.chdir('../')

In [4]:
%pwd

'D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00015_MLOps_End-to-End-ML-Application-Project_MLflow'

In [5]:
# Set up environment variables for monitoring experiments
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/soyinkas1/PP00015_MLOps_End-to-End-ML-Application-Project_MLflow.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='soyinkas1'
os.environ['MLFLOW_TRACKING_PASSWORD']='064e0bf4875a7186f622763f3cff47a14ab0f2a6'



In [6]:
# Define the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
        
        

In [7]:
# Import the needed contants and methods
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories,save_json

In [8]:
# Define the configuration manager
class ConfigurationManager:
    
    def __init__(self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH, 
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/soyinkas1/PP00015_MLOps_End-to-End-ML-Application-Project_MLflow.mlflow")
        
        return model_evaluation_config

In [9]:
# define the components
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib





[2023-10-15 21:02:35,117: INFO: utils: NumExpr defaulting to 4 threads.]


In [10]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric('rmse', rmse)
            mlflow.log_metric('r2', r2)
            mlflow.log_metric('mae', mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != 'file':
            
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # Please refer to the doc for more information
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, 'model', registered_model_name='ElasticnetModel')
            else:
                mlflow.sklearn.log_model(model, 'model')
    

In [11]:
# Create evaluation pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-15 21:02:42,825: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-15 21:02:42,835: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-15 21:02:42,843: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-15 21:02:42,846: INFO: common: created directory at: artifacts]
[2023-10-15 21:02:42,854: INFO: common: created directory at: artifacts/model_evaluation]
[2023-10-15 21:02:48,343: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


C:\Users\admin\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/10/15 21:03:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
